stellargraph is only compatible with python 3.8 so we downgrade python
then we need to re-install pip and import some other libraries to be able to install stellargraph with pip and import it

In [27]:
!pip install chardet

In [28]:
pip install stellargraph

Note: you may need to restart the kernel to use updated packages.


In [29]:
import pandas as pd
import os
import keras
import stellargraph as sg
from stellargraph.mapper import FullBatchNodeGenerator
from stellargraph.layer import GCN
from stellargraph.layer import GAT
from stellargraph.mapper import GraphSAGENodeGenerator
from stellargraph.layer import GraphSAGE
from stellargraph.datasets import Cora
from stellargraph import StellarGraph
from tensorflow.keras import layers, optimizers, losses, metrics, Model,regularizers
from sklearn import preprocessing, model_selection
from IPython.display import display, HTML
import matplotlib.pyplot as plt
%matplotlib inline
import networkx as nx
from keras.initializers import GlorotUniform
import numpy as np
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping,ModelCheckpoint

# Data Prep

building the cora. stellargraph from the dataset instead of using the built-in cora tocostumize the node feautures 

In [30]:
cora = Cora()
cora.download()

# the base_directory property tells us where it was downloaded to:
cora_cites_file = os.path.join(cora.base_directory, "cora.cites")
#we need make changes to cora.content
#a row of cora.content: ID+1433 o or 1+ label
cora_content_file = os.path.join(cora.base_directory, "cora.content")

 **loading nodes**

We’ve now got the files on disk, so we can read them using the pd.read_csv function. Despite the “CSV” in the name, this function can be used to read TSV files too. The files don’t have a row of column headings, so we’ll want to set our own.

First, the edges. We can use source and target as the column headings, to match StellarGraph’s defaults. However, the natural phrasing is “paper X cites paper Y”, not “paper Y is cited by paper X”, so we use the columns in reverse order to match.

In [31]:
cora_cites = pd.read_csv(
    cora_cites_file,
    sep="\t",  # tab-separated
    header=None,  # no heading row
    names=["target", "source"],  # set our own names for the columns
)
cora_cites

target   source
0         35     1033
1         35   103482
2         35   103515
3         35  1050679
4         35  1103960
...      ...      ...
5424  853116    19621
5425  853116   853155
5426  853118  1140289
5427  853155   853118
5428  954315  1155073

[5429 rows x 2 columns]

**loading features**

creating the features only using graph information

Now, the nodes. Again, we have to choose the columns’ names. The names of the 0/1-columns don’t matter so much, but we can give the first column (of IDs) and the last one (of subjects) useful names.

In [32]:
cora_feature_names = [f"w{i}" for i in range(1433)]

cora_raw_content = pd.read_csv(
    cora_content_file,
    sep="\t",  # tab-separated
    header=None,  # no heading row
    names=["id", *cora_feature_names, "subject"],  # set our own names for the columns
)
cora_raw_content

id  w0  w1  w2  w3  w4  w5  w6  w7  w8  ...  w1424  w1425  w1426  \
0       31336   0   0   0   0   0   0   0   0   0  ...      0      0      1   
1     1061127   0   0   0   0   0   0   0   0   0  ...      0      1      0   
2     1106406   0   0   0   0   0   0   0   0   0  ...      0      0      0   
3       13195   0   0   0   0   0   0   0   0   0  ...      0      0      0   
4       37879   0   0   0   0   0   0   0   0   0  ...      0      0      0   
...       ...  ..  ..  ..  ..  ..  ..  ..  ..  ..  ...    ...    ...    ...   
2703  1128975   0   0   0   0   0   0   0   0   0  ...      0      0      0   
2704  1128977   0   0   0   0   0   0   0   0   0  ...      0      0      0   
2705  1128978   0   0   0   0   0   0   0   0   0  ...      0      0      0   
2706   117328   0   0   0   0   1   0   0   0   0  ...      0      0      0   
2707    24043   0   0   0   0   0   0   0   0   0  ...      0      0      0   

      w1427  w1428  w1429  w1430  w1431  w1432                 subject  
0         0      0      0      0      0      0         Neural_Networks  
1         0      0      0      0      0      0           Rule_Learning  
2         0      0      0      0      0      0  Reinforcement_Learning  
3         0      0      0      0      0      0  Reinforcement_Learning  
4         0      0      0      0      0      0   Probabilistic_Methods  
...     ...    ...    ...    ...    ...    ...                     ...  
2703      0      0      0      0      0      0      Genetic_Algorithms  
2704      0      0      0      0      0      0      Genetic_Algorithms  
2705      0      0      0      0      0      0      Genetic_Algorithms  
2706      0      0      0      0      0      0              Case_Based  
2707      0      0      0      0      0      0         Neural_Networks  

[2708 rows x 1435 columns]

As we saw above when adding node features, StellarGraph uses the index of the DataFrame as the connection between a node and a row of the DataFrame. Currently our dataframe just has a simple numeric range as the index, but it needs to be using the id column. Pandas offers a few ways to control the indexing; in this case, we want to replace the current index by moving the id column to it, which is done most easily with set_index:

In [33]:
cora_content_str_subject = cora_raw_content.set_index("id")
cora_content_str_subject

w0  w1  w2  w3  w4  w5  w6  w7  w8  w9  ...  w1424  w1425  w1426  \
id                                               ...                        
31336     0   0   0   0   0   0   0   0   0   0  ...      0      0      1   
1061127   0   0   0   0   0   0   0   0   0   0  ...      0      1      0   
1106406   0   0   0   0   0   0   0   0   0   0  ...      0      0      0   
13195     0   0   0   0   0   0   0   0   0   0  ...      0      0      0   
37879     0   0   0   0   0   0   0   0   0   0  ...      0      0      0   
...      ..  ..  ..  ..  ..  ..  ..  ..  ..  ..  ...    ...    ...    ...   
1128975   0   0   0   0   0   0   0   0   0   0  ...      0      0      0   
1128977   0   0   0   0   0   0   0   0   0   0  ...      0      0      0   
1128978   0   0   0   0   0   0   0   0   0   0  ...      0      0      0   
117328    0   0   0   0   1   0   0   0   0   0  ...      0      0      0   
24043     0   0   0   0   0   0   0   0   0   0  ...      0      0      0   

         w1427  w1428  w1429  w1430  w1431  w1432                 subject  
id                                                                         
31336        0      0      0      0      0      0         Neural_Networks  
1061127      0      0      0      0      0      0           Rule_Learning  
1106406      0      0      0      0      0      0  Reinforcement_Learning  
13195        0      0      0      0      0      0  Reinforcement_Learning  
37879        0      0      0      0      0      0   Probabilistic_Methods  
...        ...    ...    ...    ...    ...    ...                     ...  
1128975      0      0      0      0      0      0      Genetic_Algorithms  
1128977      0      0      0      0      0      0      Genetic_Algorithms  
1128978      0      0      0      0      0      0      Genetic_Algorithms  
117328       0      0      0      0      0      0              Case_Based  
24043        0      0      0      0      0      0         Neural_Networks  

[2708 rows x 1434 columns]

remove subject from content

In [34]:
cora_content_no_subject = cora_content_str_subject.drop(columns="subject")
cora_content_no_subject

w0  w1  w2  w3  w4  w5  w6  w7  w8  w9  ...  w1423  w1424  w1425  \
id                                               ...                        
31336     0   0   0   0   0   0   0   0   0   0  ...      0      0      0   
1061127   0   0   0   0   0   0   0   0   0   0  ...      0      0      1   
1106406   0   0   0   0   0   0   0   0   0   0  ...      0      0      0   
13195     0   0   0   0   0   0   0   0   0   0  ...      0      0      0   
37879     0   0   0   0   0   0   0   0   0   0  ...      0      0      0   
...      ..  ..  ..  ..  ..  ..  ..  ..  ..  ..  ...    ...    ...    ...   
1128975   0   0   0   0   0   0   0   0   0   0  ...      0      0      0   
1128977   0   0   0   0   0   0   0   0   0   0  ...      0      0      0   
1128978   0   0   0   0   0   0   0   0   0   0  ...      0      0      0   
117328    0   0   0   0   1   0   0   0   0   0  ...      1      0      0   
24043     0   0   0   0   0   0   0   0   0   0  ...      0      0      0   

         w1426  w1427  w1428  w1429  w1430  w1431  w1432  
id                                                        
31336        1      0      0      0      0      0      0  
1061127      0      0      0      0      0      0      0  
1106406      0      0      0      0      0      0      0  
13195        0      0      0      0      0      0      0  
37879        0      0      0      0      0      0      0  
...        ...    ...    ...    ...    ...    ...    ...  
1128975      0      0      0      0      0      0      0  
1128977      0      0      0      0      0      0      0  
1128978      0      0      0      0      0      0      0  
117328       0      0      0      0      0      0      0  
24043        0      0      0      0      0      0      0  

[2708 rows x 1433 columns]

the newe feature vector

we will use graph algorithms
first column is the degree of each node

In [35]:
#df=pd.DataFrame()
#df['id']=cora_raw_content['id']
g = StellarGraph({"paper": cora_content_no_subject}, {"cites": cora_cites})
#print(g.info())
degree=g.node_degrees()
#print(degree)
df = pd.DataFrame.from_dict(degree, orient='index', columns=['degree'])
df.index.name='id'

In [36]:
df

degree
id             
31336         5
1061127       1
1106406       4
13195         3
37879         1
...         ...
1128975       3
1128977       2
1128978       4
117328        4
24043         3

[2708 rows x 1 columns]

second column is page rank (we need to convert graph to networkx

Pagerank is an algorithm that measures the importance or influence of nodes in a graph. It was originally developed to rank the importance of web pages in the early days of the internet.
The idea behind pagerank is that the importance of a node is proportional to the sum of the importance of the nodes that link to it. The more important nodes that link to a node, the more important that node is considered to be.
Pagerank is calculated iteratively. At each iteration, the importance of each node is updated based on the importance of the nodes that link to it. The process continues until the importance scores of the nodes converge to a stable value.

3rd column is the clustering coefficient

The clustering coefficient of a node in a graph is a measure of how closely its neighbors are connected to each other. More specifically, it measures the fraction of the node's neighbors that are also neighbors of each other.

The clustering coefficient of a single node indicates how much the node's local neighborhood is clustered. If the clustering coefficient of a node is high, it means that its neighbors are well connected to each other, forming a tightly knit cluster. On the other hand, if the clustering coefficient of a node is low, it means that its neighbors are not well connected to each other, and the node's neighborhood is more dispersed.

In general, nodes with high clustering coefficients are often considered to be part of a community or a tightly knit group of nodes, while nodes with low clustering coefficients are often considered to be more isolated or peripheral to the network. However, it's important to note that the interpretation of clustering coefficients depends on the specific context and characteristics of the graph.

adding node centrality

In [37]:
nx_G = nx.Graph()

# add the nodes to the networkx graph
nx_G.add_nodes_from(g.nodes())

# add the edges to the networkx graph
for edge in g.edges():
    source, target = edge
    if not nx_G.has_edge(source, target):
        nx_G.add_edge(source, target, weight=1)
    else:
        nx_G[source][target]["weight"] += 1

page_rank = nx.pagerank(nx_G)
clustering_coef = nx.clustering(nx_G)
betweenness_centrality = nx.betweenness_centrality(nx_G)
closeness_centrality = nx.closeness_centrality(nx_G)
eigenvector_centrality = nx.eigenvector_centrality(nx_G)


# store results in a DataFrame
df2 = pd.DataFrame({"page_rank":page_rank,
                    "clustering_coef":clustering_coef,
                    "betweenness_centrality": betweenness_centrality,
                   "closeness_centrality": closeness_centrality,
                   "eigenvector_centrality": eigenvector_centrality})


In [38]:
print(type(df2['betweenness_centrality'].iloc[0]))

<class 'numpy.float64'>


In [39]:
df2

page_rank  clustering_coef  betweenness_centrality  \
31336     0.000375         0.800000            1.365161e-07   
1061127   0.000121         0.000000            0.000000e+00   
1106406   0.000289         0.166667            2.034748e-04   
13195     0.000243         0.000000            2.746370e-05   
37879     0.000167         0.000000            0.000000e+00   
...            ...              ...                     ...   
1128975   0.000273         0.000000            6.946761e-06   
1128977   0.000228         0.000000            5.341290e-05   
1128978   0.000374         0.000000            3.430157e-04   
117328    0.000392         0.000000            6.803628e-04   
24043     0.000376         0.000000            6.960027e-04   

         closeness_centrality  eigenvector_centrality  
31336                0.133507                0.000005  
1061127              0.142318                0.000036  
1106406              0.180401                0.004253  
13195                0.160564                0.000479  
37879                0.132421                0.000008  
...                       ...                     ...  
1128975              0.154053                0.004501  
1128977              0.135984                0.000389  
1128978              0.150493                0.004558  
117328               0.130406                0.000049  
24043                0.133703                0.000005  

[2708 rows x 5 columns]

fixing betweenness values

In [40]:
df2['betweenness_centrality2'] = df2['betweenness_centrality'].apply(lambda y: format(y, '.6f'))
df2['betweenness_centrality2'] = df2['betweenness_centrality2'].apply(float)

In [41]:
print(type(df2['betweenness_centrality2'].iloc[0]))

<class 'numpy.float64'>


In [42]:
df2

page_rank  clustering_coef  betweenness_centrality  \
31336     0.000375         0.800000            1.365161e-07   
1061127   0.000121         0.000000            0.000000e+00   
1106406   0.000289         0.166667            2.034748e-04   
13195     0.000243         0.000000            2.746370e-05   
37879     0.000167         0.000000            0.000000e+00   
...            ...              ...                     ...   
1128975   0.000273         0.000000            6.946761e-06   
1128977   0.000228         0.000000            5.341290e-05   
1128978   0.000374         0.000000            3.430157e-04   
117328    0.000392         0.000000            6.803628e-04   
24043     0.000376         0.000000            6.960027e-04   

         closeness_centrality  eigenvector_centrality  betweenness_centrality2  
31336                0.133507                0.000005                 0.000000  
1061127              0.142318                0.000036                 0.000000  
1106406              0.180401                0.004253                 0.000203  
13195                0.160564                0.000479                 0.000027  
37879                0.132421                0.000008                 0.000000  
...                       ...                     ...                      ...  
1128975              0.154053                0.004501                 0.000007  
1128977              0.135984                0.000389                 0.000053  
1128978              0.150493                0.004558                 0.000343  
117328               0.130406                0.000049                 0.000680  
24043                0.133703                0.000005                 0.000696  

[2708 rows x 6 columns]

normalizing

In [43]:
def normalize_column(df, column_name):
    """
    Normalizes a column of a Pandas DataFrame between 0 and 1.
    Args:
        df (pandas.DataFrame): The Pandas DataFrame that contains the column to be normalized.
        column_name (str): The name of the column to be normalized.
    Returns:
        pandas.Series: The normalized values of the column.
    """
    column = df[column_name]
    normalized_column = (column - column.min()) / (column.max() - column.min())
    return normalized_column

df3 = pd.DataFrame()

# normalize the column using the `normalize` method
df3['norm_deg'] = normalize_column(df,'degree')
df3['norm_page_rank']=normalize_column(df2,'page_rank')
df3['norm_cluster_coef']=normalize_column(df2,'clustering_coef')
df3['norm_betweennes']=normalize_column(df2,'betweenness_centrality2')
df3['norm_closeness']=normalize_column(df2,'closeness_centrality')
df3['norm_eigenvector']=normalize_column(df2,'eigenvector_centrality')



In [44]:
print(max(df3['norm_betweennes']))

1.0


In [45]:
df3

norm_deg  norm_page_rank  norm_cluster_coef  norm_betweennes  \
id                                                                      
31336    0.023810        0.022337           0.800000         0.000000   
1061127  0.000000        0.001132           0.000000         0.000000   
1106406  0.017857        0.015170           0.166667         0.000873   
13195    0.011905        0.011314           0.000000         0.000116   
37879    0.000000        0.004957           0.000000         0.000000   
...           ...             ...                ...              ...   
1128975  0.011905        0.013874           0.000000         0.000030   
1128977  0.005952        0.010090           0.000000         0.000228   
1128978  0.017857        0.022335           0.000000         0.001475   
117328   0.017857        0.023836           0.000000         0.002925   
24043    0.011905        0.022495           0.000000         0.002994   

         norm_closeness  norm_eigenvector  
id                                         
31336          0.598643          0.000007  
1061127        0.638261          0.000055  
1106406        0.809498          0.006501  
13195          0.720303          0.000732  
37879          0.593761          0.000012  
...                 ...               ...  
1128975        0.691026          0.006878  
1128977        0.609781          0.000595  
1128978        0.675019          0.006966  
117328         0.584699          0.000074  
24043          0.599524          0.000008  

[2708 rows x 6 columns]

In [46]:
df2.drop('betweenness_centrality',axis=1)

page_rank  clustering_coef  closeness_centrality  \
31336     0.000375         0.800000              0.133507   
1061127   0.000121         0.000000              0.142318   
1106406   0.000289         0.166667              0.180401   
13195     0.000243         0.000000              0.160564   
37879     0.000167         0.000000              0.132421   
...            ...              ...                   ...   
1128975   0.000273         0.000000              0.154053   
1128977   0.000228         0.000000              0.135984   
1128978   0.000374         0.000000              0.150493   
117328    0.000392         0.000000              0.130406   
24043     0.000376         0.000000              0.133703   

         eigenvector_centrality  betweenness_centrality2  
31336                  0.000005                 0.000000  
1061127                0.000036                 0.000000  
1106406                0.004253                 0.000203  
13195                  0.000479                 0.000027  
37879                  0.000008                 0.000000  
...                         ...                      ...  
1128975                0.004501                 0.000007  
1128977                0.000389                 0.000053  
1128978                0.004558                 0.000343  
117328                 0.000049                 0.000680  
24043                  0.000005                 0.000696  

[2708 rows x 5 columns]

In [47]:
#z-score
#from sklearn.preprocessing import StandardScaler
#scaler = StandardScaler()
# normalize the entire dataframe
#df4 = pd.DataFrame(scaler.fit_transform(df2), columns=df2.columns,index=df2.index)


In [48]:
#df4

In [49]:
cora_subject = cora_content_str_subject["subject"]
cora_subject

id
31336             Neural_Networks
1061127             Rule_Learning
1106406    Reinforcement_Learning
13195      Reinforcement_Learning
37879       Probabilistic_Methods
                    ...          
1128975        Genetic_Algorithms
1128977        Genetic_Algorithms
1128978        Genetic_Algorithms
117328                 Case_Based
24043             Neural_Networks
Name: subject, Length: 2708, dtype: object

We’ve got all the right node data, and the right edges, so now we can create a StellarGraph using the techniques we saw in the “homogeneous graph with features” section.

In [50]:
#aug

result = pd.concat([cora_content_no_subject, df3], axis=1)


In [51]:
result

w0  w1  w2  w3  w4  w5  w6  w7  w8  w9  ...  w1429  w1430  w1431  \
id                                               ...                        
31336     0   0   0   0   0   0   0   0   0   0  ...      0      0      0   
1061127   0   0   0   0   0   0   0   0   0   0  ...      0      0      0   
1106406   0   0   0   0   0   0   0   0   0   0  ...      0      0      0   
13195     0   0   0   0   0   0   0   0   0   0  ...      0      0      0   
37879     0   0   0   0   0   0   0   0   0   0  ...      0      0      0   
...      ..  ..  ..  ..  ..  ..  ..  ..  ..  ..  ...    ...    ...    ...   
1128975   0   0   0   0   0   0   0   0   0   0  ...      0      0      0   
1128977   0   0   0   0   0   0   0   0   0   0  ...      0      0      0   
1128978   0   0   0   0   0   0   0   0   0   0  ...      0      0      0   
117328    0   0   0   0   1   0   0   0   0   0  ...      0      0      0   
24043     0   0   0   0   0   0   0   0   0   0  ...      0      0      0   

         w1432  norm_deg  norm_page_rank  norm_cluster_coef  norm_betweennes  \
id                                                                             
31336        0  0.023810        0.022337           0.800000         0.000000   
1061127      0  0.000000        0.001132           0.000000         0.000000   
1106406      0  0.017857        0.015170           0.166667         0.000873   
13195        0  0.011905        0.011314           0.000000         0.000116   
37879        0  0.000000        0.004957           0.000000         0.000000   
...        ...       ...             ...                ...              ...   
1128975      0  0.011905        0.013874           0.000000         0.000030   
1128977      0  0.005952        0.010090           0.000000         0.000228   
1128978      0  0.017857        0.022335           0.000000         0.001475   
117328       0  0.017857        0.023836           0.000000         0.002925   
24043        0  0.011905        0.022495           0.000000         0.002994   

         norm_closeness  norm_eigenvector  
id                                         
31336          0.598643          0.000007  
1061127        0.638261          0.000055  
1106406        0.809498          0.006501  
13195          0.720303          0.000732  
37879          0.593761          0.000012  
...                 ...               ...  
1128975        0.691026          0.006878  
1128977        0.609781          0.000595  
1128978        0.675019          0.006966  
117328         0.584699          0.000074  
24043          0.599524          0.000008  

[2708 rows x 1439 columns]

In [52]:
cora_no_subject = StellarGraph({"paper": result}, {"cites": cora_cites})
print(cora_no_subject.info())

StellarGraph: Undirected multigraph
 Nodes: 2708, Edges: 5429

 Node types:
  paper: [2708]
    Features: float32 vector, length 1439
    Edge types: paper-cites->paper

 Edge types:
    paper-cites->paper: [5429]
        Weights: all 1 (default)
        Features: none


In [53]:
print(len(cora_cites))

5429


renaming--**rest follows like library**

In [54]:
G=cora_no_subject
node_subjects=cora_subject

In [60]:
def my_gcn(G,node_subjects):
    #splitting the data : 140 nodes for training, 500 for validation and 2068 for testing
    train_subjects, test_subjects = model_selection.train_test_split(node_subjects, train_size=140, test_size=None, stratify=node_subjects)
    val_subjects, test_subjects = model_selection.train_test_split(test_subjects, train_size=500, test_size=None, stratify=test_subjects)
    #one hot encoding for node subjects
    target_encoding = preprocessing.LabelBinarizer()
    train_targets = target_encoding.fit_transform(train_subjects)
    val_targets = target_encoding.transform(val_subjects)
    test_targets = target_encoding.transform(test_subjects)
    #creating GCN layers
    #using full batch node generator to convert graph structure and node features into a format that can be fed into a keras model
    generator = FullBatchNodeGenerator(G, method="gcn")
    #matching nodes and actual labels to create model input
    train_gen = generator.flow(train_subjects.index, train_targets)
    #specifying the machine learning model: GCN with 2 hidden layers (16 units each) Relu for activation function dropout rate 50%
    gcn = GCN(layer_sizes=[16, 16], activations=["relu", "relu"], generator=generator, dropout=0.5)
    #input and output tensors
    # x_out is <KerasTensor: shape=(1, None, 16) dtype=float32 (created by layer 'gather_indices_1')>
    x_inp, x_out = gcn.in_out_tensors()
    #Predictions of each node’s class/subject will be computed from x_out (tensor that holds a 16-dim vector for nodes requested in traing/prediction)
    #using an additional dense layer (with one unit per class) with softmax activation.
    #softmax ensures that the final outputs for each input node will be a vector of probabilities( all values between 0 and 1, vector sum is 1).
    #Predicted class is the element with the highest value.
    initializer = keras.initializers.GlorotUniform()
    predictions = layers.Dense(units=train_targets.shape[1], activation="softmax",kernel_initializer=initializer)(x_out)
    #training
    #creating the Keras model (to compute/predict 16 dim node embedding vecors, we can swap predictions to x_out and use dim reduction for visualizing)
    
    model = Model(inputs=x_inp, outputs=predictions)
    model.compile(
    optimizer=optimizers.Adam(learning_rate=0.01),
    loss=losses.categorical_crossentropy,
    metrics=["acc",metrics.Precision(),metrics.Recall()],
    )
    #validation and test generator
    val_gen = generator.flow(val_subjects.index, val_targets)
    test_gen = generator.flow(test_subjects.index, test_targets)
    #stop training when validation accuracy stops improving
    es_callback = EarlyStopping(monitor="val_acc", patience=50, restore_best_weights=True)
    #training the model
    history = model.fit(
    train_gen,
    epochs=200,
    validation_data=val_gen,
    verbose=2,
    shuffle=False,  # this should be False, since shuffling data means shuffling the whole graph
    callbacks=[es_callback],
    )
    #testing
    #metrics for trained model
    #The keras.evaluate() function will give you the loss value for every batch. The keras.predict() function will give you the actual predictions for all samples in a batch, for all batches.
    test_metrics = model.evaluate(test_gen)
    for name, val in zip(model.metrics_names, test_metrics):
        print("\t{}: {:0.4f}".format(name, val))
    if test_metrics[2]+test_metrics[3]==0:
        f1_score='inf'
    else:
        f1_score=(2*test_metrics[2]*test_metrics[3])/(test_metrics[2]+test_metrics[3])
    print("f1_score:",f1_score)
    #making predictions
    #predictiong all classes without knowing the subject
    all_nodes = node_subjects.index
    all_gen = generator.flow(all_nodes)
    all_predictions = model.predict(all_gen)
    #inverse transform from one-hot encoding
    node_predictions = target_encoding.inverse_transform(all_predictions.squeeze())
    df = pd.DataFrame({"Predicted": node_predictions, "True": node_subjects})
    print(df)


def my_sgn(G,node_subjects):
    train_subjects, test_subjects = model_selection.train_test_split(
        node_subjects, train_size=140, test_size=None, stratify=node_subjects
    )
    val_subjects, test_subjects = model_selection.train_test_split(
        test_subjects, train_size=500, test_size=None, stratify=test_subjects
    )
    target_encoding = preprocessing.LabelBinarizer()

    train_targets = target_encoding.fit_transform(train_subjects)
    val_targets = target_encoding.transform(val_subjects)
    test_targets = target_encoding.transform(test_subjects)
    generator = FullBatchNodeGenerator(G, method="sgc", k=2)
    train_gen = generator.flow(train_subjects.index, train_targets)
    sgc = GCN(
        layer_sizes=[train_targets.shape[1]],
        generator=generator,
        bias=True,
        dropout=0.5,
        activations=["softmax"],
        kernel_regularizer=regularizers.l2(5e-4),
    )
    x_inp, predictions = sgc.in_out_tensors()
    model = Model(inputs=x_inp, outputs=predictions)
    model.compile(
        optimizer=optimizers.Adam(lr=0.2),
        loss=losses.categorical_crossentropy,
        metrics=["acc",metrics.Precision(),metrics.Recall()],
    )
    val_gen = generator.flow(val_subjects.index, val_targets)
    if not os.path.isdir("logs"):
        os.makedirs("logs")
    es_callback = EarlyStopping(
        monitor="val_acc", patience=50
    )  # patience is the number of epochs to wait before early stopping in case of no further improvement
    mc_callback = ModelCheckpoint(
        "logs/best_model_sgn.h5", monitor="val_acc", save_best_only=True, save_weights_only=True
    )
    history = model.fit(
        train_gen,
        epochs=50,
        validation_data=val_gen,
        verbose=0,
        shuffle=False,  # this should be False, since shuffling data means shuffling the whole graph
        callbacks=[es_callback, mc_callback],
    )
    model.load_weights("logs/best_model_sgn.h5")
    test_gen = generator.flow(test_subjects.index, test_targets)
    test_metrics = model.evaluate(test_gen)
    for name, val in zip(model.metrics_names, test_metrics):
        print("\t{}: {:0.4f}".format(name, val))
    if test_metrics[2]+test_metrics[3]==0:
        f1_score='inf'
    else:
        f1_score=(2*test_metrics[2]*test_metrics[3])/(test_metrics[2]+test_metrics[3])
    print("f1_score:",f1_score)

def my_gat(G,node_subjects):
    train_subjects, test_subjects = model_selection.train_test_split(
        node_subjects, train_size=140, test_size=None, stratify=node_subjects
    )
    val_subjects, test_subjects = model_selection.train_test_split(
        test_subjects, train_size=500, test_size=None, stratify=test_subjects
    )
    target_encoding = preprocessing.LabelBinarizer()

    train_targets = target_encoding.fit_transform(train_subjects)
    val_targets = target_encoding.transform(val_subjects)
    test_targets = target_encoding.transform(test_subjects)
    generator = FullBatchNodeGenerator(G, method="gat")
    train_gen = generator.flow(train_subjects.index, train_targets)
    gat = GAT(
        layer_sizes=[8, train_targets.shape[1]],
        activations=["elu", "softmax"],
        attn_heads=8,
        generator=generator,
        in_dropout=0.5,
        attn_dropout=0.5,
        normalize=None,
    )
    x_inp, predictions = gat.in_out_tensors()
    model = Model(inputs=x_inp, outputs=predictions)
    model.compile(
        optimizer=optimizers.Adam(lr=0.005),
        loss=losses.categorical_crossentropy,
        metrics=["acc",metrics.Precision(),metrics.Recall()],
    )
    val_gen = generator.flow(val_subjects.index, val_targets)
    if not os.path.isdir("logs"):
        os.makedirs("logs")
    es_callback = EarlyStopping(
        monitor="val_acc", patience=20
    )  # patience is the number of epochs to wait before early stopping in case of no further improvement
    mc_callback = ModelCheckpoint(
        "logs/best_model_gat.h5", monitor="val_acc", save_best_only=True, save_weights_only=True
    )
    history = model.fit(
        train_gen,
        epochs=50,
        validation_data=val_gen,
        verbose=2,
        shuffle=False,  # this should be False, since shuffling data means shuffling the whole graph
        callbacks=[es_callback, mc_callback],
    )
    model.load_weights("logs/best_model_gat.h5")
    test_gen = generator.flow(test_subjects.index, test_targets)
    test_metrics = model.evaluate(test_gen)
    for name, val in zip(model.metrics_names, test_metrics):
        print("\t{}: {:0.4f}".format(name, val))
    if test_metrics[2]+test_metrics[3]==0:
        f1_score='inf'
    else:
        f1_score=(2*test_metrics[2]*test_metrics[3])/(test_metrics[2]+test_metrics[3])
    print("f1_score:",f1_score)
    
def my_graphsage(G,node_subjects):
    train_subjects, test_subjects = model_selection.train_test_split(node_subjects, train_size=0.1, test_size=None, stratify=node_subjects)
    target_encoding = preprocessing.LabelBinarizer()

    train_targets = target_encoding.fit_transform(train_subjects)
    test_targets = target_encoding.transform(test_subjects)
    batch_size = 50
    num_samples = [10, 5]
    generator = GraphSAGENodeGenerator(G, batch_size, num_samples)
    train_gen = generator.flow(train_subjects.index, train_targets, shuffle=True)
    graphsage_model = GraphSAGE(
    layer_sizes=[32, 32], generator=generator, bias=True, dropout=0.5,
    )
    x_inp, x_out = graphsage_model.in_out_tensors()
    prediction = layers.Dense(units=train_targets.shape[1], activation="softmax")(x_out)
    model = Model(inputs=x_inp, outputs=prediction)
    model.compile(
        optimizer=optimizers.Adam(lr=0.005),
        loss=losses.categorical_crossentropy,
        metrics=["acc",metrics.Precision(),metrics.Recall()],
    )
    test_gen = generator.flow(test_subjects.index, test_targets)
    history = model.fit(
        train_gen, epochs=20, validation_data=test_gen, verbose=2, shuffle=False
    )
    test_metrics = model.evaluate(test_gen)
    for name, val in zip(model.metrics_names, test_metrics):
        print("\t{}: {:0.4f}".format(name, val))
    if test_metrics[2]+test_metrics[3]==0:
        f1_score='inf'
    else:
        f1_score=(2*test_metrics[2]*test_metrics[3])/(test_metrics[2]+test_metrics[3])
    print("f1_score:",f1_score)
    #test_metrics = model.evaluate(test_gen)
    #print("\nTest Set Metrics:")
    #for name, val in zip(model.metrics_names, test_metrics):
        #print("\t{}: {:0.4f}".format(name, val))


In [61]:
my_gcn(G,node_subjects)

Using GCN (local pooling) filters...
Epoch 1/200
1/1 - 2s - loss: 1.9505 - acc: 0.1071 - precision_4: 0.0000e+00 - recall_4: 0.0000e+00 - val_loss: 1.8830 - val_acc: 0.3140 - val_precision_4: 0.0000e+00 - val_recall_4: 0.0000e+00 - 2s/epoch - 2s/step
Epoch 2/200
1/1 - 0s - loss: 1.8692 - acc: 0.3071 - precision_4: 0.0000e+00 - recall_4: 0.0000e+00 - val_loss: 1.8127 - val_acc: 0.3020 - val_precision_4: 0.0000e+00 - val_recall_4: 0.0000e+00 - 132ms/epoch - 132ms/step
Epoch 3/200
1/1 - 0s - loss: 1.7758 - acc: 0.3214 - precision_4: 1.0000 - recall_4: 0.0071 - val_loss: 1.7459 - val_acc: 0.3020 - val_precision_4: 0.0000e+00 - val_recall_4: 0.0000e+00 - 128ms/epoch - 128ms/step
Epoch 4/200
1/1 - 0s - loss: 1.6871 - acc: 0.3000 - precision_4: 1.0000 - recall_4: 0.0143 - val_loss: 1.6972 - val_acc: 0.3020 - val_precision_4: 0.6429 - val_recall_4: 0.0180 - 131ms/epoch - 131ms/step
Epoch 5/200
1/1 - 0s - loss: 1.6077 - acc: 0.3143 - precision_4: 0.8824 - recall_4: 0.1071 - val_loss: 1.6441 - v

In [65]:
my_sgn(G,node_subjects)

Calculating 2-th power of normalized A...
1/1 [==============================] - 0s 112ms/step - loss: 1.6304 - acc: 0.4753 - precision_8: 0.8000 - recall_8: 0.0019
	loss: 1.6304
	acc: 0.4753
	precision_8: 0.8000
	recall_8: 0.0019
f1_score: 0.0038591414509943135


In [63]:
my_gat(G,node_subjects)

/opt/conda/lib/python3.7/site-packages/keras/initializers/initializers_v2.py:121: UserWarning: The initializer GlorotUniform is unseeded and being called multiple times, which will return identical values  each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initalizer instance more than once.
  f"The initializer {self.__class__.__name__} is unseeded "


Epoch 1/50
1/1 - 7s - loss: 1.9841 - acc: 0.1571 - precision_6: 0.0000e+00 - recall_6: 0.0000e+00 - val_loss: 1.9236 - val_acc: 0.2420 - val_precision_6: 0.0000e+00 - val_recall_6: 0.0000e+00 - 7s/epoch - 7s/step
Epoch 2/50
1/1 - 0s - loss: 1.9457 - acc: 0.1357 - precision_6: 0.0000e+00 - recall_6: 0.0000e+00 - val_loss: 1.9007 - val_acc: 0.3040 - val_precision_6: 0.0000e+00 - val_recall_6: 0.0000e+00 - 234ms/epoch - 234ms/step
Epoch 3/50
1/1 - 0s - loss: 1.8989 - acc: 0.2214 - precision_6: 0.0000e+00 - recall_6: 0.0000e+00 - val_loss: 1.8791 - val_acc: 0.3440 - val_precision_6: 0.0000e+00 - val_recall_6: 0.0000e+00 - 227ms/epoch - 227ms/step
Epoch 4/50
1/1 - 0s - loss: 1.9025 - acc: 0.2286 - precision_6: 0.0000e+00 - recall_6: 0.0000e+00 - val_loss: 1.8581 - val_acc: 0.3640 - val_precision_6: 0.0000e+00 - val_recall_6: 0.0000e+00 - 259ms/epoch - 259ms/step
Epoch 5/50
1/1 - 0s - loss: 1.8606 - acc: 0.2714 - precision_6: 0.0000e+00 - recall_6: 0.0000e+00 - val_loss: 1.8373 - val_acc: 0.

In [64]:
my_graphsage(G,node_subjects)

/opt/conda/lib/python3.7/site-packages/keras/initializers/initializers_v2.py:121: UserWarning: The initializer GlorotUniform is unseeded and being called multiple times, which will return identical values  each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initalizer instance more than once.
  f"The initializer {self.__class__.__name__} is unseeded "


Epoch 1/20
6/6 - 4s - loss: 1.9011 - acc: 0.1963 - precision_7: 0.0000e+00 - recall_7: 0.0000e+00 - val_loss: 1.8140 - val_acc: 0.3265 - val_precision_7: 0.0000e+00 - val_recall_7: 0.0000e+00 - 4s/epoch - 667ms/step
Epoch 2/20
6/6 - 2s - loss: 1.8194 - acc: 0.3185 - precision_7: 0.0000e+00 - recall_7: 0.0000e+00 - val_loss: 1.7576 - val_acc: 0.3179 - val_precision_7: 0.0000e+00 - val_recall_7: 0.0000e+00 - 2s/epoch - 282ms/step
Epoch 3/20
6/6 - 3s - loss: 1.7603 - acc: 0.3519 - precision_7: 0.0000e+00 - recall_7: 0.0000e+00 - val_loss: 1.7183 - val_acc: 0.3244 - val_precision_7: 0.0000e+00 - val_recall_7: 0.0000e+00 - 3s/epoch - 475ms/step
Epoch 4/20
6/6 - 2s - loss: 1.7203 - acc: 0.3852 - precision_7: 0.0000e+00 - recall_7: 0.0000e+00 - val_loss: 1.6772 - val_acc: 0.3417 - val_precision_7: 0.0000e+00 - val_recall_7: 0.0000e+00 - 2s/epoch - 304ms/step
Epoch 5/20
6/6 - 3s - loss: 1.6521 - acc: 0.4481 - precision_7: 0.0000e+00 - recall_7: 0.0000e+00 - val_loss: 1.6324 - val_acc: 0.3802 -